# Redes Neuronales en PyTorch para regresión
Este notebook entrena una red neuronal sencilla para predecir el precio medio de las casas en California utilizando la base de datos [CaliforniaHousing](CaliforniaHousing).

In [ ]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader


## Carga y preparación de datos

In [ ]:
data_path = 'CaliforniaHousing/cal_housing.data'
data = np.loadtxt(data_path, delimiter=',')
X = data[:, :-1]
y = data[:, -1:]
# División entrenamiento/prueba
rng = np.random.default_rng(42)
indices = rng.permutation(len(X))
split = int(len(X) * 0.8)
train_idx, test_idx = indices[:split], indices[split:]
X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]
# Estandarización
X_mean, X_std = X_train.mean(axis=0), X_train.std(axis=0)
y_mean, y_std = y_train.mean(axis=0), y_train.std(axis=0)
X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std
y_train = (y_train - y_mean) / y_std
y_test = (y_test - y_mean) / y_std
# Tensores y DataLoader
tensor_X_train = torch.tensor(X_train, dtype=torch.float32)
tensor_y_train = torch.tensor(y_train, dtype=torch.float32)
tensor_X_test = torch.tensor(X_test, dtype=torch.float32)
tensor_y_test = torch.tensor(y_test, dtype=torch.float32)
train_dataset = TensorDataset(tensor_X_train, tensor_y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


## Definición del modelo

In [ ]:
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


## Entrenamiento

In [ ]:
n_epochs = 20
losses = []
for epoch in range(n_epochs):
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    losses.append(avg_loss)
    if (epoch+1) % 5 == 0:
        print(f'Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}')


## Evaluación

In [ ]:
model.eval()
with torch.no_grad():
    preds = model(tensor_X_test)
    mse = criterion(preds, tensor_y_test).item()
    preds_orig = preds.numpy() * y_std + y_mean
    y_test_orig = tensor_y_test.numpy() * y_std + y_mean
    rmse = ((preds_orig - y_test_orig) ** 2).mean() ** 0.5
print(f'Final Test MSE (scaled): {mse:.4f}')
print(f'Final Test RMSE: {rmse:.2f}')
